### Obtiene la lista de todos los investigadores que aparecen en la página de renacit

In [1]:
####################################
## A. Librerias
####################################
import os
import time
import json

import pandas as pd

from utils.utils import get_chrome_driver,stop_chrome_driver

from utils.utils import TIME_SLEEP
from utils.paths import CHROMEDRIVER_PATH, PROJECT_PATH


####################################
## B. Funciones
####################################

def get_files(path, extension):
    dir_name  = os.path.join(os.getcwd(), path)
    files     = os.listdir(dir_name)
    
    paths    = []
    for file in files:
        if extension in file:
            paths.append(os.path.join(path,file))

    return paths

def start_har(proxy, name):
    options = {
        'captureHeaders': True, 
        'captureContent': True, 
        'captureBinaryContent': True, 
        'captureCookies': True, 
        'captureHeadersSize': -1, 
        'captureMaxSize': -1, 
        'captureBinaryContentMaxLength': -1 # Tamaño máximo del contenido binario
    }
    proxy.new_har(name, options=options)

ESTAMOS EN --->  windows
PROJECT_PATH:  C:\GitHubProjets\RenacytScraper
CHROME_DRIVER_PATH:  C:\GitHubProjets\RenacytScraper\chromedriver\windows\chromedriver.exe



In [2]:
########################
## C. Ejecución
########################

# 1. Seleccionar modo
HEADLESS = False
PRINT_VIEW = False

# 2. Eliminar todos los archivos de la carpeta
try:
    files = get_files(os.path.join(os.getcwd(),'originals','renacyt', 'investigadores'), '.')
    for file in files:
        os.remove(file)
    print('Se eliminaron los archivos de la carpeta')
except:
    pass

Se eliminaron los archivos de la carpeta


In [3]:
######################
## D. Ejecución
######################

# 0. Definir los datos de la cuenta
url = 'https://servicio-renacyt.concytec.gob.pe/busqueda-de-investigadores/'

# 1. Iniciar el driver, el servidor proxy y el proxy
driver, server, proxy = get_chrome_driver(chromedriver_path=CHROMEDRIVER_PATH, print_view=PRINT_VIEW, headless=HEADLESS)

# 2. Iniciar har
start_har(proxy, 'renacyt')

# 3. Abrir la página
driver.get(url)

Se inicia el servidor en Windows
Usando el chromedriver_path: C:\GitHubProjets\RenacytScraper\chromedriver\windows\chromedriver.exe


In [4]:
# 4. Obtener el .har y guardar los json
time.sleep(TIME_SLEEP*2)
har = proxy.har

# 5. Procesar el .har
i = 0
for entrie in har.get('log').get('entries'):
    aux_entrie = str(entrie)

    if '"codigoRegistro"' in aux_entrie and '"institucionLaboralPrincipal"' in aux_entrie:
        string_content = entrie.get('response').get('content').get('text')
        data_list = json.loads(string_content)
        with open(os.path.join(os.getcwd(),'originals','renacyt','investigadores',str(i).zfill(2) + '.json'), 'w') as json_file:
            json.dump(data_list, json_file, indent=4)
        i += 1

# 6. Cerrar el driver, el servidor proxy y el proxy
stop_chrome_driver(driver, server, proxy)

# 7. Eliminar los archivos que no tienen datos
try:
    os.remove('bmp.log')
    os.remove('server.log')
    print("Se eliminaron los archivos bmp.log y server.log")
except:
    pass